# **TextTorch - 06: Deploy e Inferência**

**Objetivo:** Demonstrar como usar o modelo treinado para fazer predições em novos textos. Este notebook simula um ambiente de "deploy" onde carregamos os artefatos salvos (modelo e vectorizer) para inferência.

### **Como Executar (Google Colab)**

1. **Pré-requisito:** Execute os notebooks `01` a `05` para gerar todos os artefatos.
2. **Ambiente:** Se estiver em um novo ambiente, clone o repositório e instale as dependências.
   ```bash
   !git clone https://github.com/takaokensei/TextTorch.git
   %cd TextTorch
   !pip install -r requirements.txt
   ```
3. **Execução:** Execute todas as células.

In [2]:
# Imports e Configurações Iniciais
import sys
import os

# Adiciona o diretório 'src' ao path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from deploy import Predictor, save_inference_example
from model import load_config

# Carrega configurações
CONFIG_PATH = '../models/config.yaml'
config = load_config(CONFIG_PATH)

print("Ambiente configurado.")

2025-11-13 17:32:37,077 - INFO - Configuração carregada de: ../models/config.yaml


Ambiente configurado.


## 1. Carregando o Preditor

A classe `Predictor` do módulo `src/deploy.py` encapsula toda a lógica necessária para a inferência:
- Carrega o modelo treinado.
- Carrega o `TfidfVectorizer`.
- Preprocessa o texto de entrada.
- Realiza a predição.

In [3]:
# Caminhos para os artefatos
MODEL_PATH = f"../models/{config['representation']}_model.pt"
VECTORIZER_PATH = '../artifacts/vectorizer.joblib'

# Instancia o preditor
try:
    predictor = Predictor(model_path=MODEL_PATH, vectorizer_path=VECTORIZER_PATH)
    print("Preditor carregado com sucesso!")
except FileNotFoundError as e:
    print(f"ERRO: {e}")
    print("\nCertifique-se de que os notebooks anteriores foram executados e os artefatos foram salvos.")

2025-11-13 17:32:40,562 - INFO - Inicializando preditor...
2025-11-13 17:32:40,570 - INFO - TFIDFClassifier criado: input=676, hidden=512, output=6
2025-11-13 17:32:40,572 - INFO - Modelo carregado de: ../models/tfidf_model.pt
2025-11-13 17:32:40,581 - INFO - Preditor inicializado com sucesso.


Preditor carregado com sucesso!


## 2. Teste Interativo de Predição

Agora, podemos usar o objeto `predictor` para classificar novos textos.

In [4]:
# Lista de textos para testar
textos_exemplo = [
    "The new graphics card from Nvidia offers amazing performance for gaming.", # Computação/Gráficos
    "Scientists are debating the existence of dark matter in the universe.", # Ciência/Espaço
    "O governo anunciou novas medidas para controlar a inflação.", # Política/Economia (em português)
    "This email is an urgent security alert for your account.", # Spam/Alerta
    "How can I fix this error in my Python code?", # Programação
    "The best hockey players will compete for the Stanley Cup.", # Esportes
    "A discussão sobre a reforma tributária continua no congresso.", # Política (em português)
]

for texto in textos_exemplo:
    try:
        label, prob = predictor.predict(texto)
        print(f"Texto: '{texto[:60]}...'\n  -> Predição: {label} (Confiança: {prob:.2%})\n")
    except Exception as e:
        print(f"Erro ao predizer o texto: '{texto[:60]}...'. Causa: {e}")

Texto: 'The new graphics card from Nvidia offers amazing performance...'
  -> Predição: Classe 1 (Confiança: 48.73%)

Texto: 'Scientists are debating the existence of dark matter in the ...'
  -> Predição: Classe 1 (Confiança: 19.43%)

Texto: 'O governo anunciou novas medidas para controlar a inflação....'
  -> Predição: Classe 1 (Confiança: 23.51%)

Texto: 'This email is an urgent security alert for your account....'
  -> Predição: Classe 1 (Confiança: 19.43%)

Texto: 'How can I fix this error in my Python code?...'
  -> Predição: Classe 1 (Confiança: 19.43%)

Texto: 'The best hockey players will compete for the Stanley Cup....'
  -> Predição: Classe 1 (Confiança: 19.43%)

Texto: 'A discussão sobre a reforma tributária continua no congresso...'
  -> Predição: Classe 4 (Confiança: 26.01%)



## 3. Salvando Exemplo de Inferência

Para documentar o projeto, salvamos um arquivo de texto com os resultados das predições de exemplo.

In [5]:
INFERENCE_EXAMPLE_PATH = '../reports/example_inference.txt'

try:
    save_inference_example(predictor, textos_exemplo, save_path=INFERENCE_EXAMPLE_PATH)
except NameError:
    print("Preditor não foi carregado. Pulando salvamento de exemplo.")

2025-11-13 17:33:02,895 - INFO - Exemplo de inferência salvo em: ../reports/example_inference.txt
